In [23]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
pd.set_option('display.max_columns', None)

ImportError: cannot import name 'OrderedModel' from 'statsmodels.discrete.discrete_model' (c:\Anaconda 3\Lib\site-packages\statsmodels\discrete\discrete_model.py)

In [3]:
### prepare data
df_t = pd.read_excel('data.xlsx')
df_t.columns = df_t.columns.str.strip()
# random value
# new_df = pd.DataFrame({col: np.random.choice(df[col], size=500) for col in df.columns})

In [4]:
df = df_t
df = df.iloc[57:]

### save or load model

In [ ]:
### save model
with open('MNlogit.pkl', 'wb') as f:  # ใช้ 'wb' เพื่อใช้โหมด binary
    pickle.dump(MNLogit, f)
### load model
with open('MNlogit.pkl', 'rb') as f:  # ใช้ 'rb' เพื่อใช้โหมด binary
    loaded_model = pickle.load(f)

### clean data

In [5]:
### 2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง
condition = [df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].isin(['1-3 ครั้ง','1 - 4 ครั้ง']),df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].isin(['ไม่ได้รับประทานเลย'])]
df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] =  np.select(condition,['1 - 4 ครั้ง','ไม่ได้รับประทานเลย'] , default='มากกว่า 4 ครั้ง')
df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].unique()

### 1.4 อายุ
condition = [df['1.4 อายุ'].isin(['21-30 ปี','21 - 30 ปี']),df['1.4 อายุ'].isin(['31-40 ปี','31 - 40 ปี']),df['1.4 อายุ'].isin(['41 - 50 ปี','51 ปีขึ้นไป'])]
df['1.4 อายุ'] =  np.select(condition,['21 - 30 ปี','31 - 40 ปี','40 ปีขึ้นไป'],default='ต่ำกว่า 20 ปี')
df['1.4 อายุ'].unique()

#1.6 อาชีพ
condition = [df['1.6 อาชีพ'].isin(['ข้าราชการ/รัฐวิสาหกิจ','พนักงานของรัฐ','พนักงานรัฐ','ลูกจ้างของหน่วยงานรัฐ','ลูกจ้างชั่วคราว ภาครัฐ','ลูกจ้างหน่วยงานรัฐ','ลูกจ้างหน่วยงานราชการ','ลูกจ้างในหน่วยงานราชการ '])
             ,df['1.6 อาชีพ'].isin(['พนักงานบริษัทเอกชน'])]
df['1.6 อาชีพ'] =  np.select(condition,['ข้าราชการ/รัฐวิสาหกิจ','พนักงานบริษัทเอกชน'],default='อื่น')
df['1.6 อาชีพ'].unique()

### หัวข้อ 3
df['3.1 product_hat'] = (df['3.1 ด้านผลิตภัณฑ์ (product)  [อาหารมีความหลากลาย]']+df['3.1 ด้านผลิตภัณฑ์ (product)  [รสชาติของอาหาร]']+df['3.1 ด้านผลิตภัณฑ์ (product)  [ความสะอาดของอาหาร]'])/3
df['3.2 Price_hat'] = (df['3.2 ด้านราคา (price)  [ราคาที่เหมาะสมกับคุณภาพ]']+df['3.2 ด้านราคา (price)  [ราคาของอาหารเหมาะสมเมื่อเทียบกับร้านอื่น]'])/2
df['3.3 Place_hat'] = (df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [เวลาเปิด-ปิดของร้านในการให้บริการ]']+df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [ความสะดวกของที่จอดรถ]']+df['3.3 ด้านช่องทางการจัดจำหน่าย (Place)  [ระยะทาง]'])/3
df['3.4 Promotion_hat'] = (df['3.4 ด้านโปรโมชั่น (promotion)  [ส่วนลดหรือโปรโมชั่นส่งเสริมการขายในช่วงเทศกาลต่าง ๆ เช่น เทศกาลขึ้นปีใหม่]']+df['3.4 ด้านโปรโมชั่น (promotion)  [การสื่อสารข้อมูลโปรโมชั่นผ่านช่องทางต่างๆ เช่น YouTube, Facebook, TikTok]']+df['3.4 ด้านโปรโมชั่น (promotion)  [การมอบสิทธิพิเศษกับสมาชิก]'])/3
df['3.5 People_hat'] = (df['3.5 ด้านบุคลากร (people) [พนักงานมี Service Mind ที่ดี]']+df['3.5 ด้านบุคลากร (people) [ความสะอาดในการแต่งกายของพนักงาน]']+df['3.5 ด้านบุคลากร (people) [จำนวนพนักงานมีเพียงพอในการให้บริการ]'])/3
df['3.6 process_hat'] = (df['3.6 ด้านกระบวนการ (process)  [การจัดลำดับคิวการให้บริการลูกค้า]']+df['3.6 ด้านกระบวนการ (process)  [กระบวนการสั่งเครื่องดื่มและอาหาร การรับคำสั่ง และการเสิร์ฟเครื่องดื่มและอาหารตามที่สั่ง]']+df['3.6 ด้านกระบวนการ (process)  [ระยะเวลาในการรออาหาร]'])/3

### 16. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู
condition = [df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].isin(['ลดราคา']),df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].isin(['กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค'])]
df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'] =  np.select(condition,['ลดราคา','กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค'] , default='อื่น ๆ')
df['6. กลยุทธ์ส่งเสริมการขายแบบใดที่จะทำให้ท่านทานบุฟเฟต์ชาบู'].unique()

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_11816\1135786901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] =  np.select(condition,['1 - 4 ครั้ง','ไม่ได้รับประทานเลย'] , default='มากกว่า 4 ครั้ง')
C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_11816\1135786901.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['1.4 อายุ'] =  np.select(condition,['21 - 30 ปี','31 - 40 ปี','40 ปีขึ้นไป'],default='ต่ำกว่า 20 ปี')
C:\Users\chaiyaphonp\AppDat

array(['ลดราคา', 'กลยุทธ์ส่งเสริมการขายไม่มีผลต่อการตัดสินใจเลือกบริโภค',
       'อื่น ๆ'], dtype=object)

In [15]:
# col_name = df.columns
# for col,i in zip(col_name,range(0,len(col_name))):
#     df.rename(columns={col:'x_{}'.format(i)},inplace=True)
#     n = pd.DataFrame(data={'var':df.columns,'col_name':col_name})

# with pd.ExcelWriter("data_adjust3.xlsx") as writer:
   
#     # use to_excel function and specify the sheet_name and index 
#     # to store the dataframe in specified sheet
#     df.to_excel(writer, sheet_name="data", index=False)
#     n.to_excel(writer, sheet_name="col_name", index=False)    

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_20656\1685574391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={col:'x_{}'.format(i)},inplace=True)


### MNlogit model

In [64]:
col_y = ['5. หากให้ท่านเลือกรับประทานบุฟเฟต์ชาบูได้ 1 แบบ\nท่านจะเลือกแบบใด']
# col_y =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ,'2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง','2.2 เหตุผลใดที่ทำให้ท่านเลือกที่จะรับประทานบุฟเฟต์ชาบู'
#         ,'2.3 ส่วนใหญ่ท่านรับประทานบุฟเฟต์ชาบูกับใคร','2.4 โดยส่วนใหญ่ราคาบุฟเฟต์ชาบูต่อหัวที่ท่านเลือกรับประทานอยู่ที่เท่าไร']
col_x =[ '3.1 product_hat' ,'3.2 Price_hat' ,'3.3 Place_hat' ,'3.4 Promotion_hat' ,'3.5 People_hat' ,'3.6 process_hat']
col_x_dummy = ['1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ]
y = df[col_y]
x = df[col_x]
x_dummy = df[col_x_dummy]
# y = y[~y.isnull().any(axis=1)]
# x = x[~x.isnull().any(axis=1)]
x_dummy = x_dummy[~x_dummy.isnull().any(axis=1)]
# y = new_df[col_x]
# x = new_df[col_y]
# for clean data
# y = pd.get_dummies(y,dtype=float)
# y.drop(['1.1 ท่านเคยรับประทานบุปเฟต์ชาบูหรือไม่_ไม่เคย'],axis=1,inplace=True)
# x_dummy = pd.get_dummies(x_dummy,dtype=float,drop_first=True)
x_dummy = pd.get_dummies(x_dummy,dtype=float)
x_dummy = x_dummy[['1.3 เพศ_หญิง', '1.4 อายุ_21 - 30 ปี','1.4 อายุ_31 - 40 ปี', '1.4 อายุ_40 ปีขึ้นไป','1.5 รายได้เฉลี่ยต่อเดือน_100,000 บาทขึ้นไป',
       '1.5 รายได้เฉลี่ยต่อเดือน_12,000 - 15,000 บาท','1.5 รายได้เฉลี่ยต่อเดือน_15,001 - 35,000 บาท','1.5 รายได้เฉลี่ยต่อเดือน_35,001 - 50,000 บาท',
       '1.5 รายได้เฉลี่ยต่อเดือน_50,001 - 100,000 บาท','1.6 อาชีพ_ข้าราชการ/รัฐวิสาหกิจ', '1.6 อาชีพ_พนักงานบริษัทเอกชน']]
x = pd.concat([x, x_dummy],axis = 1)
# x = sm.add_constant(x)
# x_dummy = sm.add_constant(x_dummy)
# MNlogit model
model = sm.MNLogit(y, x)
MNLogit = model.fit()
print(MNLogit.summary())

         Current function value: 1.308141
         Iterations: 35
                                                 MNLogit Regression Results                                                 
Dep. Variable:     5. หากให้ท่านเลือกรับประทานบุฟเฟต์ชาบูได้ 1 แบบ
ท่านจะเลือกแบบใด   No. Observations:                  176
Model:                                                                      MNLogit   Df Residuals:                      108
Method:                                                                         MLE   Df Model:                           64
Date:                                                              Tue, 30 Apr 2024   Pseudo R-squ.:                  0.1180
Time:                                                                      10:29:45   Log-Likelihood:                -230.23
converged:                                                                    False   LL-Null:                       -261.05
Covariance Type:                                           

c:\Anaconda 3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [6]:
coef = pd.DataFrame(data=result.params)
coef.rename(columns = {0:'coef_B',1:'coef_C',2:'coef_D',3:'coef_E'}, inplace = True)
P_value = pd.DataFrame(data=result.pvalues)
P_value.rename(columns = {0:'P_value_B',1:'P_value_C',2:'P_value_D',3:'P_value_E'}, inplace = True)
all = pd.concat([P_value, coef],axis = 1)
all
# all.to_excel('result.xlsx')

## OrderedModel

In [60]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from pandas.api.types import CategoricalDtype
## define y,x variable
col_y = ['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง']
# col_y =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ' ,'2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง','2.2 เหตุผลใดที่ทำให้ท่านเลือกที่จะรับประทานบุฟเฟต์ชาบู'
#         ,'2.3 ส่วนใหญ่ท่านรับประทานบุฟเฟต์ชาบูกับใคร','2.4 โดยส่วนใหญ่ราคาบุฟเฟต์ชาบูต่อหัวที่ท่านเลือกรับประทานอยู่ที่เท่าไร']
col_x =[ '1.3 เพศ','1.4 อายุ', '1.5 รายได้เฉลี่ยต่อเดือน', '1.6 อาชีพ']
y = df[col_y]
# y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].replace({'1 - 4 ครั้ง':1, 'มากกว่า 4 ครั้ง':2, 'ไม่ได้รับประทานเลย':0},inplace=True)
x = df[col_x]
cat_type = CategoricalDtype(categories=['ไม่ได้รับประทานเลย', '1 - 4 ครั้ง', 'มากกว่า 4 ครั้ง'], ordered=True)
y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] = y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].astype(cat_type)
# y = new_df[col_x]
# x = new_df[col_y]
# for clean data
# y = pd.get_dummies(y,dtype=float)
# y.drop(['1.1 ท่านเคยรับประทานบุปเฟต์ชาบูหรือไม่_ไม่เคย'],axis=1,inplace=True)
# x = pd.get_dummies(x,dtype=float,drop_first=True)
# # MNlogit model
model = OrderedModel(y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'], x,distr='logit')
# result = model.fit()
# print(result.summary())

C:\Users\chaiyaphonp\AppData\Local\Temp\ipykernel_11816\1703617674.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'] = y['2.1 ในเดือนที่ผ่านมาท่านทานบุฟเฟต์ชาบูกี่ครั้ง'].astype(cat_type)


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).